In [ ]:
import pandas
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import tkinter
from sko.PSO import PSO
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(1)#控制变量

In [ ]:
names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
HeartDisease = pandas.read_csv('./processed.cleveland.csv',header=None,names=names)

In [ ]:
median=HeartDisease['ca'].median()
HeartDisease['ca'].fillna(median,inplace=True)
median=HeartDisease['thal'].median()
HeartDisease['thal'].fillna(median,inplace=True)
#HeartDisease.info()

In [ ]:
label = HeartDisease['num']
data = HeartDisease.drop(['num'],axis=1)

In [ ]:
if data.shape[0] == label.shape[0]:
    print('Sample number is equal to label number')
else:
    print('Sample number not equal to label number')
    exit(-1)

In [ ]:
clf=SVC(kernel='rbf')#测试不同svm在不同核函数下的性能
acc  = cross_val_score(clf, data,label, cv=3)
print(np.mean(acc))

In [ ]:
clf = SVC(kernel='poly')#测试不同svm在不同核函数下的性能
acc = cross_val_score(clf, data, label, cv=3)
print(np.mean(acc))

In [ ]:
clf = SVC(kernel='sigmoid')#测试不同svm在不同核函数下的性能
acc = cross_val_score(clf, data, label, cv=3)
print(np.mean(acc))

In [ ]:
clf = SVC(kernel='linear')#测试不同svm在不同核函数下的性能
acc = cross_val_score(clf, data, label, cv=3)
print(np.mean(acc))

In [ ]:
x_train , x_test , y_train , y_test =train_test_split(data, label, test_size=0.25, train_size=0.75, random_state=42)

In [ ]:
def plot(position):
    x = []
    y = []
    for i in range(0,len(position)):
        x.append(position[i][0])
        y.append(position[i][1])
    colors = (0,0,0)
    plt.scatter(x, y, c = colors, alpha = 0.1)
    plt.xlabel('C')
    plt.ylabel('gamma')
    plt.axis([0,1,0,1])
    plt.gca().set_aspect('equal', adjustable='box')
    return plt.show()

In [ ]:
class var:
    count=0
    bestsocre=0.0
    bestpos=[]

In [ ]:
def func(x):
    x1,x2=x
    clf = SVC
    socre = cross_val_score(clf(kernel='linear',C=x1, gamma=x2),data, label, cv=3).mean()
    if(socre>var.bestsocre):
        var.bestsocre=socre
        var.bestpos=x
    if var.count%10==0:
        print('iter', int(var.count / 10))
    print(x,socre)
    if var.count%10==9:
        print('best position is',var.bestpos,'best score is',var.bestsocre)
        var.bestsocre=0
    var.count+=1
    return 1-socre

In [ ]:
pso = PSO(func=func, n_dim=2, pop=10, max_iter=50, lb=[0.001, 0.01], ub=[1, 10], w=0.8, c1=0.5, c2=0.5)
pso.record_mode = True
pso.run()
print('best_C and best_gamma is ', pso.gbest_x, 'best_cost is', pso.gbest_y)

In [ ]:
clf=SVC(kernel='linear',C=pso.gbest_x[0],gamma=pso.gbest_x[1])
psoacc  = cross_val_score(clf, data,label, cv=3).mean()
print(psoacc)

In [ ]:
if data.shape[0] == label.shape[0]:
    root = tkinter.Tk()
    text = tkinter.Text(root,width=20,height=1)
    text.insert('1.0',psoacc)
    text.pack()
    root.title('ops-svm预测准确率')
    root.geometry('240x120')
    button = tkinter.Button(root, text="确定", command=root.destroy)
    button.place(x=100,y=80)
    root.mainloop()
else:
    root = tkinter.Tk()
    text = tkinter.Text(root,width=20,height=1)
    text.insert('1.0','发生错误')
    text.pack()
    root.title('预测准确率')
    root.geometry('240x120')
    button = tkinter.Button(root, text="确定", command=root.destroy)
    button.place(x=100,y=80)
    root.mainloop()